# ყველა ფაილის მისამართის ამოღება

In [1]:
!pip install librosa
!pip install pydub

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [29]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         spectral_features = get_spectral_features(audio, sample_rate)
        chroma_features = get_chroma_features(audio, sample_rate)
#         other features
#         return [mfccs, zero_crossing_rate] + spectral_features + chroma_features
        return [mfccs]
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [30]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
#     data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate', 'spectral_features[0]', 'spectral_features[1]', 'spectral_features[2]',  'chroma_features[0]','chroma_features[1]', 'chroma_features[2]', 'class_label'])
    data = pd.DataFrame(all_features, columns=['mfccs', 'class_label'])
    return data

In [31]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [6]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [7]:
def get_spectral_features(audio, sr):
    sp = librosa.feature.spectral_centroid(audio)
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    sp_scaled = np.mean(sp.T,axis=0)
    a_scaled = np.mean(a.T,axis=0)
    onset_env_scaled = np.mean(onset_env,axis=0)
    return [sp_scaled, a_scaled, onset_env_scaled]

In [7]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [9]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [32]:
data = get_all_features('data')

In [33]:
data

,mfccs,class_label
0,"[-375.04742, 71.76099, -9.058271, 58.62941, -2...",1
1,"[-602.8608, 40.34575, 19.960438, 24.787498, 6....",1
2,"[-551.16956, 59.801514, -0.14260375, 24.958803...",1
3,"[-512.303, 39.143753, -3.8368192, 26.026289, 1...",1
4,"[-583.33325, 23.309853, 2.6051655, 16.292341, ...",1
...,...,...
683,"[-428.7131, 72.103806, 8.8743925, 34.941143, 1...",5
684,"[-505.51352, 74.166504, 19.460085, 22.2807, -2...",5
685,"[-353.27307, 102.400566, 6.734233, 49.48563, -...",5
686,"[-622.4822, 107.89334, 3.9235218, 20.481346, 0...",5


In [34]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.1, random_state = 127)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(619, 2) (69, 2) (619, 1) (69, 1)


In [35]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [36]:
y_onehot_train = get_one_hot(y_train)
y_onehot_test = get_one_hot(y_test)
print(y_onehot_train.shape, y_onehot_test.shape)

(619, 5) (69, 5)


/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "cate

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [37]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
    return res

In [38]:
ls_train = get_processed_X(x_train)
ls_test = get_processed_X(x_test)
print(ls_train.shape, ls_test.shape)

(619, 40) (69, 40)


 # ნეირონული ქსელის კოდი

In [15]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [16]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [17]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = []
    for i in range(z3.shape[0]):
        z = [z3[i, j] for j in range(z3.shape[1])]
        h.append(softmax(z))
    h = np.array(h)
#     h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [18]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

#     J = (-1 / m) * np.sum(y * np.log(h).T) #softmax
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [19]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [20]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = cost(params,input_size, hidden_size, num_labels, X, y, learning_rate)

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [39]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 25
num_labels = y_onehot_train.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((25, 41), (5, 26))

In [40]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape
print(h)
print(cost(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate))

[[0.28754724 0.14424646 0.25488318 0.22312675 0.09019637]
 [0.28134257 0.14555026 0.25241203 0.22803542 0.09265972]
 [0.28307371 0.14379572 0.25262825 0.2287861  0.09171621]
 ...
 [0.27859901 0.14364651 0.2507537  0.23360173 0.09339905]
 [0.28047743 0.1442734  0.25180215 0.23071275 0.09273426]
 [0.2945373  0.14436087 0.25742944 0.21618313 0.08748925]]
2.5906122853542723


In [41]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(2.5906122853542723, (1155,))

In [43]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


     fun: 1.7344200416738422
     jac: array([-1.33121816e-07, -6.24516522e-05, -8.36702166e-05, ...,
       -3.15494328e-03, -3.24853279e-03, -3.05937101e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 30
  status: 3
 success: False
       x: array([ 0.02610821, -0.05529028, -0.04455745, ...,  0.02456713,
       -0.03244276,  0.08405872])

In [44]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
y_pred_train = np.array(np.argmax(h, axis=1) + 1)

In [45]:
ls_test = np.matrix(ls_test)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

In [46]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.5767366720516963


In [47]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.4492753623188406
